# dMRI Preprocessing

This note book processes multiband diffusion weighted imaging for tensor-based analyses

In [ ]:
from nipype.pipeline.engine import Workflow, Node, JoinNode, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from nipype.interfaces.fsl import BET, MeanImage, ApplyTOPUP, TOPUP, Merge,ExtractROI

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# Study-specific variables
project_home = '/data/perlman/moochie/user_data/CamachoCat/ChEC/dmri_proc'
output_dir = project_home + '/proc/preprocessing'
workflow_dir = project_home + '/workflows'
raw_dir = '/data/perlman/moochie/study_data/ChEC/MRI_data'
phase_encoding_file = project_home + '/misc/chec_encoding_file.txt'

subjects_list = open(project_home + '/misc/chec_subjects.txt').read().splitlines()

# FreeSurfer set up - change SUBJECTS_DIR 
fs_dir = '/moochie/Cat/Aggregate_anats/subjects_dir'
FSCommand.set_default_subjects_dir(fs_dir)

In [55]:
### Universal nodes

# get subjects list
infosource = Node(IdentityInterface(fields=['subjid']),
                  name='infosource')
infosource.iterables = [('subjid', subjects_list)]

# Sink data of interest
substitutions = [('_subjid_', '')] #output file name substitutions
datasink = Node(DataSink(base_directory = output_dir,
                        container = output_dir,
                        substitutions = substitutions), 
                name='datasink')

## Unwarping workflow

This workflow includes both unwarping and eddy correction.

In [57]:
# select dMRI data
dmri_template = {'dmri_pe0': raw_dir + '/sub-{subjid}/dwi/sub-{subjid}_98dir_AP.nii.gz',
                 'bval':raw_dir + '/sub-{subjid}/dwi/sub-{subjid}_98dir_AP.bval',
                 'bvec':raw_dir + '/sub-{subjid}/dwi/sub-{subjid}_98dir_AP.bvec'}
selectdmri = Node(SelectFiles(dmri_template), name='selectdmri')

pes_template={'pes':raw_dir + '/sub-{subjid}/dwi/sub-{subjid}_98dir_{pe}.nii.gz'}
selectpes = Node(SelectFiles(pes_template),name='selectpes')
selectpes.iterables=('pe',['AP','PA'])

# include only the first volume of each PE volume
trim_pes = Node(ExtractROI(t_min=0,t_size=1,roi_file='pe_trimmed.nii.gz'),name='trim_pes')

# merge to 1 file for topup to calculate the fieldcoef
merge_pes = JoinNode(Merge(dimension='t',
                           merged_file='merged_pes.nii.gz'),
                     name='merge_pes', joinsource='selectpes', joinfield='in_files')

# actually do the unwarping
topup = Node(TOPUP(encoding_file=phase_encoding_file), name='topup')

apply_topup = Node(ApplyTOPUP(in_index=[1], encoding_file=phase_encoding_file,
                              method='jac', out_corrected='dmri_unwarped.nii.gz'),
                   name='apply_topup')

# average dMRI so we can make a mask
avg_dmri = Node(MeanImage(), name='avg_dmri')

# create a dMRI space brain mask using BET
make_mask = Node(BET(mask=True), name='make_mask')

In [58]:
prepreprocflow = Workflow(name='prepreprocflow')
prepreprocflow.connect([(infosource,selectdmri, [('subjid','subjid')]),
                        (infosource, selectpes, [('subjid','subjid')]),
                        (selectpes, trim_pes, [('pes','in_file')]),
                        (trim_pes, merge_pes,[('roi_file','in_files')]),
                        (merge_pes, topup, [('merged_file','in_file')]),
                        (topup, apply_topup, [('out_fieldcoef','in_topup_fieldcoef'), 
                                              ('out_movpar','in_topup_movpar')]),
                        (selectdmri, apply_topup, [('dmri_pe0','in_files')]),
                        (apply_topup, avg_dmri, [('out_corrected','in_file')]),
                        (avg_dmri, make_mask, [('out_file','in_file')]),
                        (make_mask, datasink, [('mask_file','in_mask')]),
                        (apply_topup, datasink, [('out_corrected','unwarped_niftis')])
                       ])

prepreprocflow.base_dir = workflow_dir
#prepreprocflow.write_graph(graph2use='flat')
prepreprocflow.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})

200415-10:42:39,833 nipype.workflow INFO:
	 Workflow prepreprocflow settings: ['check', 'execution', 'logging', 'monitoring']
200415-10:42:42,43 nipype.workflow INFO:
	 Running in parallel.
200415-10:42:42,51 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 78 jobs ready. Free memory (GB): 10.00/10.00, Free processors: 4/4.
200415-10:42:42,201 nipype.workflow INFO:
	 [Node] Setting-up "prepreprocflow.selectpes" in "/data/perlman/moochie/user_data/CamachoCat/ChEC/dmri_proc/workflows/prepreprocflow/_subjid_1055/_pe_PA/selectpes".
200415-10:42:42,203 nipype.workflow INFO:
	 [Node] Setting-up "prepreprocflow.selectpes" in "/data/perlman/moochie/user_data/CamachoCat/ChEC/dmri_proc/workflows/prepreprocflow/_subjid_1055/_pe_AP/selectpes".
200415-10:42:42,205 nipype.workflow INFO:
	 [Node] Setting-up "prepreprocflow.selectdmri" in "/data/perlman/moochie/user_data/CamachoCat/ChEC/dmri_proc/workflows/prepreprocflow/_subjid_1055/selectdmri".
200415-10:42:42,208 nipype.workflow INFO:
	 [No

RuntimeError: Workflow did not execute cleanly. Check log for details

## Preprocessing Workflow

This workflow performs the following steps:
* coregistration between dMRI and T1w anat
* apply warp to atlas to put atlas in subject space

In [ ]:
# FreeSurferSource - Data grabber specific for FreeSurfer data
fssource = Node(FreeSurferSource(subjects_dir=fs_dir),
                run_without_submitting=True,
                name='fssource')

def FSid(subjid):
    from nipype import logging, config
    config.enable_debug_mode()
    logging.update_logging(config)
    
    fs_id = 'C'+str(subjid)
    return(fs_id)

fsid = Node(Function(input_names=['subjid'],
                     output_names=['fs_id'], 
                     function=FSid), 
            name='fsid')

def isolate_ToM(dtk_nii):
    from nipype import logging, config
    config.enable_debug_mode()
    logging.update_logging(config)
    from nibabel import load, save, Nifti1Image
    from numpy import zeros_like
    from os.path import abspath
    
    # r precuneus, l precuneus, r supramarginal, l supramarginal, r IPL, l IPL, l rostral-anterior cing, r rostral-anterior cing, l mofc, r mofc
    labels = [2025,1025,2031,1031,2008,1008,2026,1026,2014,1014] 
    
    aparc_nii = load(dtk_nii)
    aparc_data = aparc_nii.get_data()
    
    ToM_data = zeros_like(aseg_data) 
    for x in labels:
        ToM_data[aparc_data == x] = x
    ToM_nifti = Nifti1Image(ToM_data, aparc_nii.affine)
    save(ToM_nifti, 'ToM_rois.nii.gz')
    ToM_file = abspath('ToM_rois.nii.gz')
    
    return(ToM_file)

In [ ]:
## Nodes for creating ROI volumes from freesurfer segmentation
file_template = {'dMRI': output_dir + '/unwarped_funcs/{subjid}/dmri_unwarped_reoriented.nii.gz'}
selectfiles = Node(SelectFiles(file_template),name='selectfiles')

# trim to B0 volume
trim_b0 = Node(ExtractROI(t_min=0, t_size=1), name='trim_b0')

# convert anat to nifti
t1_to_nii = Node(MRIConvert(out_file='T1w.nii.gz',out_type='niigz'),
                 name='t1_to_nii')
# reorient anat
reorient_t1 = Node(Reorient2Std(terminal_output='file'),
                    name='reorient_t1')

#convert parc to nifti
parc_to_nii = Node(MRIConvert(out_file='aparcDTK.nii.gz',out_type='niigz'),
                   name='parc_to_nii')

#reorient parc
reorient_parc = Node(Reorient2Std(terminal_output='file'),
                     name='reorient_parc')

# register anat to B0
coreg_t1_b0 = Node(FLIRT(out_file='warp_t1.nii.gz',out_matrix_file='xfm.mat'), name='coreg_t1_b0')

# apply registration to DK parcellation
apply_reg = Node(FLIRT(out_file='warp_dtk.nii.gz',apply_xfm=True), name='apply_reg')

# strip down to mentalizing network ROIs
create_ToM_ROIs = Node(Function(input_names= ['dtk_nii'], output_names=['ToM_file'],name=isolate_ToM), 
                       name='create_ToM_ROIs')


In [ ]:
roiprocflow = Workflow(name='roiprocflow')
roiprocflow.connect([(infosource, fsid, [('subjid','subjid')]), 
                     (fsid, fssource, [('fs_id','subject_id')]), 
                     (fssource, t1_to_nii, [('brain', 'in_file')]),
                     (t1_to_nii, reorient_t1, [('out_file','in_file')]),
                     (reorient_t1, coreg_t1_b0, [('out_file','in_file')]),
                     
                     (fssource, parc_to_nii, [('aparc_aseg', 'in_file')]),
                     (parc_to_nii, reorient_parc, [('out_file','in_file')]),
                     (reorient_parc, apply_reg, [('out_file','in_file')]),
                     (coreg_t1_b0, apply_reg, [('out_matrix_file','in_matrix_file')]),
                     
                     (infosource, selectfiles, [('subjid','subjid')]),
                     (selectfiles, trim_b0, [('dMRI','in_file')]),
                     (trim_b0, coreg_t1_b0, [('roi_file','reference')]),
                     (trim_b0, apply_reg, [('roi_file','reference')]),
                     (apply_reg, create_ToM_ROIs, [('out_file','dtk_nii')]),
                     
                     (apply_reg, datasink, [('out_file','warped_DTK')]),
                     (coreg_t1_b0, datasink, [('out_file','warped_T1')]),
                     (trim_b0, datasink, [('roi_file','B0_vol_files')]),
                     (create_ToM_ROIs, [('ToM_file','ToM_ROIs')])
                    ])
roiprocflow.base_dir = workflow_dir
roiprocflow.write_graph(graph2use='flat')
roiprocflow.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})